In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, KBinsDiscretizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential 
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot
from numpy import mean
from sklearn.ensemble import StackingClassifier
from sklearn.compose import ColumnTransformer

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# Load data

In [2]:
# load data from files
X_train = pd.read_csv("/kaggle/input/titanic/train.csv")
X_submission = pd.read_csv("/kaggle/input/titanic/test.csv")
y_train = X_train['Survived']

# remove y column
X_train = X_train.drop(columns='Survived')

## Data dictionary


|Variable|Definition|Key
|----|----|----
|survival|Survival|0 = No, 1 = Yes
|pclass|Ticket class|1 = 1st, 2 = 2nd, 3 = 3rd
|sex|Sex|
|Age|Age in years|
|sibsp|# of siblings / spouses aboard the Titanic
|parch|# of parents / children aboard the Titanic
|ticket|Ticket number
|fare|Passenger fare
|cabin|Cabin number
|embarked|Port of Embarkation|C = Cherbourg, Q = Queenstown, S = Southampto

# Preprocessing

In [3]:
# to use later for submission
X_submission_passengerId = X_submission['PassengerId']

X_train = X_train.drop(columns='Name')
X_train = X_train.drop(columns='Ticket')
X_train = X_train.drop(columns='PassengerId')
X_submission = X_submission.drop(columns='Name')
X_submission = X_submission.drop(columns='Ticket')
X_submission = X_submission.drop(columns='PassengerId')


In [4]:
X_train["Age"] = X_train["Age"].fillna(X_train["Age"].mean())
X_train["Cabin"] = X_train["Cabin"].fillna(0)
X_train["Embarked"] = X_train["Embarked"].fillna('S')

X_submission["Age"] = X_submission["Age"].fillna(X_submission["Age"].mean())
X_submission["Cabin"] = X_submission["Cabin"].fillna(0)
X_submission["Embarked"] = X_submission["Embarked"].fillna('S')

encode categorical cols

In [5]:
X_submission = X_submission.fillna(0)

In [6]:
print(X_train)
print()
print(X_submission)

     Pclass     Sex        Age  SibSp  Parch     Fare Cabin Embarked
0         3    male  22.000000      1      0   7.2500     0        S
1         1  female  38.000000      1      0  71.2833   C85        C
2         3  female  26.000000      0      0   7.9250     0        S
3         1  female  35.000000      1      0  53.1000  C123        S
4         3    male  35.000000      0      0   8.0500     0        S
..      ...     ...        ...    ...    ...      ...   ...      ...
886       2    male  27.000000      0      0  13.0000     0        S
887       1  female  19.000000      0      0  30.0000   B42        S
888       3  female  29.699118      1      2  23.4500     0        S
889       1    male  26.000000      0      0  30.0000  C148        C
890       3    male  32.000000      0      0   7.7500     0        Q

[891 rows x 8 columns]

     Pclass     Sex       Age  SibSp  Parch      Fare Cabin Embarked
0         3    male  34.50000      0      0    7.8292     0        Q
1        

In [7]:
cols_to_encode = ["Sex","Cabin","Embarked","Pclass"]
enc = LabelEncoder()

for col in cols_to_encode:
    X_train[col] = X_train[col].astype('str')
    X_submission[col] = X_train[col].astype('str') 
    X_train[col] = enc.fit_transform(X_train[col] ) 
    X_submission[col] = enc.fit_transform(X_submission[col])
    
#ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[6])],remainder = 'passthrough')
#X_train = ct.fit_transform(X_train)
#X_submission = ct.fit_transform(X_submission)


print(X_train)
print()
print(X_submission)

     Pclass  Sex        Age  SibSp  Parch     Fare  Cabin  Embarked
0         2    1  22.000000      1      0   7.2500      0         2
1         0    0  38.000000      1      0  71.2833     82         0
2         2    0  26.000000      0      0   7.9250      0         2
3         0    0  35.000000      1      0  53.1000     56         2
4         2    1  35.000000      0      0   8.0500      0         2
..      ...  ...        ...    ...    ...      ...    ...       ...
886       1    1  27.000000      0      0  13.0000      0         2
887       0    0  19.000000      0      0  30.0000     31         2
888       2    0  29.699118      1      2  23.4500      0         2
889       0    1  26.000000      0      0  30.0000     61         0
890       2    1  32.000000      0      0   7.7500      0         1

[891 rows x 8 columns]

     Pclass  Sex       Age  SibSp  Parch      Fare  Cabin  Embarked
0         2    1  34.50000      0      0    7.8292      0         2
1         0    0  47.00

scaling

In [8]:
def scale_cols(df, cols, scaler):
    for col in cols:
        scaler = StandardScaler().fit(df[[col]])
        df[col] = scaler.transform(df[[col]])
        # scalers[col] = scaler

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

col_scale = ['Age', 'Fare', 'Cabin']

scale_cols(X_train, col_scale, sc)
scale_cols(X_submission, col_scale, sc)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,2,1,-0.592481,1,0,-0.502445,-0.462490,2
1,0,0,0.638789,1,0,0.786845,1.688672,0
2,2,0,-0.284663,0,0,-0.488854,-0.462490,2
3,0,0,0.407926,1,0,0.420730,1.006596,2
4,2,1,0.407926,0,0,-0.486337,-0.462490,2


discretize

In [9]:
kbinsAge = KBinsDiscretizer(n_bins=15, strategy='uniform', encode='ordinal')
X_train['Age'] = kbinsAge.fit_transform(np.array(X_train['Age']).reshape(-1,1))

kbinsFare = KBinsDiscretizer(n_bins=30, strategy='uniform', encode='ordinal')
X_train['Fare'] = kbinsFare.fit_transform(np.array(X_train['Fare']).reshape(-1,1))

kbinsAgeSubm = KBinsDiscretizer(n_bins=15, strategy='uniform', encode='ordinal')
X_submission['Age'] = kbinsAgeSubm.fit_transform(np.array(X_submission['Age']).reshape(-1,1))

kbinsFareSubm = KBinsDiscretizer(n_bins=30, strategy='uniform', encode='ordinal')
X_submission['Fare'] = kbinsFareSubm.fit_transform(np.array(X_submission['Fare']).reshape(-1,1))

In [10]:
X_train,X_test,y_train,y_test = train_test_split (X_train,y_train,test_size = 0.2)

lr = LogisticRegression(max_iter=10000)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix (y_test,y_pred)
score = accuracy_score (y_test,y_pred)

print(cm)
print(score)

[[84 18]
 [14 63]]
0.8212290502793296


# Predict and submission

In [11]:
y_pred_subm = lr.predict(X_submission)
y_pred_subm

array([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,

In [12]:
final_df = pd.DataFrame({'PassengerId': X_submission_passengerId, 'Survived': y_pred_subm})
final_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,1


In [13]:
final_df.to_csv('submission.csv', index=False)
print(" Submission saved")

 Submission saved
